In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers=[21,22,-108,31,-1,32,34,31]

In [3]:
tf_dataset=tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [4]:
for sales in tf_dataset:
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [5]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [6]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [7]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [8]:
tf_dataset=tf_dataset.filter(lambda x:x>0)

In [9]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [10]:
tf_dataset=tf_dataset.map(lambda x:x*72)

In [11]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [13]:
tf_dataset=tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

2232
1512
2232
2304
1584
2448


In [15]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[2232 1584]
[2448 2232]
[1512 2304]


In [16]:
tf_dataset2=tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset2

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [17]:
tf_dataset2=tf_dataset2.filter(lambda x:x>0).map(lambda y:y*72).shuffle(3).batch(2)

for sales in tf_dataset2.as_numpy_iterator():
    print(sales)

[1512 1584]
[2448 2232]
[2304 2232]


In [41]:
images_ds=tf.data.Dataset.list_files('PetImages/*/*',shuffle=False)

for file in images_ds.take(5):
    print(file.numpy())

b'PetImages\\cat\\0.jpg'
b'PetImages\\cat\\1.jpg'
b'PetImages\\cat\\10.jpg'
b'PetImages\\cat\\100.jpg'
b'PetImages\\cat\\101.jpg'


In [42]:
images_ds=images_ds.shuffle(200)

for file in images_ds.take(5):
    print(file.numpy())

b'PetImages\\cat\\136.jpg'
b'PetImages\\cat\\22.jpg'
b'PetImages\\cat\\151.jpg'
b'PetImages\\cat\\102.jpg'
b'PetImages\\cat\\149.jpg'


In [43]:
class_names=["cat","dog"]

In [44]:
image_count=len(images_ds)
image_count

1000

In [45]:
train_size=int(image_count*0.8)
train_ds=images_ds.take(train_size)

#skip is opposite of take it will skip the first eighty percent of the images and take the remaining
test_ds=images_ds.skip(train_size)

In [46]:
len(train_ds)

800

In [47]:
len(test_ds)

200

In [48]:
s='PetImages\\cat\\0.jpg'

print(s.split('\\')[1])
print(s.split('\\')[-2])

#last element is -1 so the previous element is -2 and so on

cat
cat


In [49]:
#convert the tensor to string and split 
def get_label(file_path):
    import os
    return tf.strings.split(file_path,os.path.sep)[-2]

In [50]:
def process_image(file_path):
    label=get_label(file_path)
    
    img=tf.io.read_file(file_path)
    img=tf.image.decode_jpeg(img)
    img=tf.image.resize(img,[128,128])
    
    return img,label
    

In [51]:
for label in train_ds.map(get_label).take(5):
    print(label)

tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor(b'cat', shape=(), dtype=string)


In [52]:
train_ds=train_ds.map(process_image)
for img,label in train_ds.take(3):
    print("image: ",img)
    print("label: ",label)

image:  tf.Tensor(
[[[ 13.035156  17.035156  16.035156]
  [ 13.035156  17.035156  16.035156]
  [ 13.521912  17.521912  16.521912]
  ...
  [ 11.769531  15.769531  14.769531]
  [  8.696228  12.696228  11.696228]
  [ 29.605469  29.605469  29.605469]]

 [[ 12.105469  16.105469  15.105469]
  [ 12.105469  16.105469  15.105469]
  [ 15.922546  19.922546  18.922546]
  ...
  [  8.734375  12.734375  11.734375]
  [ 10.932434  14.932434  13.932434]
  [ 28.53241   28.53241   28.53241 ]]

 [[ 11.175781  15.175781  14.175781]
  [ 11.175781  15.175781  14.175781]
  [ 13.394714  17.394714  16.394714]
  ...
  [  8.218933  12.218933  11.218933]
  [  8.703796  12.703796  11.703796]
  [ 29.463318  29.463318  29.463318]]

 ...

 [[136.05109  141.40265  147.75421 ]
  [124.87244  130.224    136.44922 ]
  [125.206116 130.60437  136.4643  ]
  ...
  [143.91016  147.91016  157.00354 ]
  [148.31293  152.31293  161.31293 ]
  [144.34003  149.98846  162.8103  ]]

 [[124.668884 131.66888  140.76263 ]
  [136.62732  143.

In [39]:
def scale(image,label):
    return image/255,label

In [53]:
train_ds=train_ds.map(scale)

for image,label in train_ds.take(5):
    print("image: ",image.numpy()[0][0])
    print("label: ",label.numpy())

image:  [0.12783562 0.10822778 0.0925415 ]
label:  b'cat'
image:  [0.5416219  0.55110055 0.55709136]
label:  b'cat'
image:  [0.3985141 0.3710631 0.3031403]
label:  b'cat'
image:  [0.5514694  0.54362625 0.49264586]
label:  b'cat'
image:  [0.02903694 0.11175752 0.25673062]
label:  b'cat'
